In [1]:
import module1
import nltk
import numpy as np
import pandas as pd
import time

from sklearn import preprocessing

from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
%load_ext autoreload
%autoreload 2

In [2]:
filename = 'ttk_train'
df = module1.read_df_from_csv(filename)
df_test = module1.read_df_from_csv('ttk_test_etalon')

In [17]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> 

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
         

True

In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
#we can add stop words

import pymorphy2
def tokenize(text):
    text = text.lower()
    text_list = nltk.word_tokenize(text)
    morph = pymorphy2.MorphAnalyzer()
    text_list = [word for word in text_list if word[0] != '@' and len(word) > 1 and word.isalpha()]
    text_list = [morph.parse(word)[0].normal_form for word in text_list]
    return text_list

In [4]:
#tf-idf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
j = 0
token_list = [i for i in df['text']]
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words=stop_words)
tfs = tfidf.fit_transform(token_list)

In [5]:
def df_filter(_df, _tfs, corporation):
    x_arr = []
    y_arr = []
    for i in range(len(_df)):
        label = _df[corporation][i]
        if label == '0' or label == '-1' or label == '1' or label == 0.0 or label == -1.0 or label == 1.0:
            x_arr.append(_tfs[i].toarray()[0])
            y_arr.append(int(label))
    X = np.array(x_arr)
    y = np.array(y_arr)
    print (len(X))
    print (len(y))
    return X, y

In [6]:
companies = ['mts', 'megafon', 'tele2', 'rostelecom', 'beeline', 'komstar', 'skylink']
X_train, y_train = df_filter(_df=df, _tfs=tfs, corporation=companies[0])
for companie in companies[1:]:
    X_curr, y_curr = df_filter(_df=df, _tfs=tfs, corporation=companie)
    X_train = np.append(X_train, X_curr, axis=0)
    y_train = np.append(y_train, y_curr)

2005
2005
1198
1198
310
310
520
520
989
989
3
3
12
12


In [7]:
companies = ['mts', 'megafon', 'tele2', 'rostelecom', 'beeline', 'komstar', 'skylink']
X_test, y_test = df_filter(_df=df_test, _tfs=tfs, corporation=companies[0])
for companie in companies[1:]:
    X_curr, y_curr = df_filter(_df=df_test, _tfs=tfs, corporation=companie)
    if len(X_curr) > 0:
        X_test = np.append(X_test, X_curr, axis=0)
        y_test = np.append(y_test, y_curr)

1454
1454
139
139
32
32
17
17
2530
2530
1
1
0
0


In [8]:
print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print ('SVM: ')
start_time = time.time()
model = SVC()
model.fit(X_train, y_train)
t = int(time.time() - start_time)
print ('fit time = {0}m{1}sec'.format(t // 60, t % 60))
start_time = time.time()
y_predict = model.predict(X_test)
t = int(time.time() - start_time)
print ('predict time = {0}m{1}sec'.format(t // 60, t % 60))
print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
SVM: 
fit time = 7m0sec
predict time = 5m38sec
f-macro: 0.26861258167596697
f-micro: 0.6748142822909178


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
import my_score
func= lambda x,y: my_score.f_macro(my_score.cound_diff(x,y))
print ('strange f score - {0}'.format(func(y_predict, y_test)))

strange f score - 0.4282238442822384


In [12]:
print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print ('Decision Tree: ')
start_time = time.time()
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
t = int(time.time() - start_time)
print ('fit time = {0} {1}'.format(t / 60, t % 60))
start_time = time.time()
y_predict = model.predict(X_test)
t = int(time.time() - start_time)
print ('predict time = {0} {1}'.format(t / 60, t % 60))
print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
print ('strange f score - {0}'.format(func(y_predict, y_test)))
print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print ('AdaBoost DecisionTree max_depth = 2, n = 100: ')
start_time = time.time()
model = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5), n_estimators=100,
learning_rate=1)
model.fit(X_train, y_train)
t = int(time.time() - start_time)
print ('fit time = {0} {1}'.format(t / 60, t % 60))
start_time = time.time()
y_predict = model.predict(X_test)
t = int(time.time() - start_time)
print ('predict time = {0} {1}'.format(t / 60, t % 60))
print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
print ('strange f score - {0}'.format(func(y_predict, y_test)))

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Decision Tree: 
fit time = 0.31666666666666665 19
predict time = 0.0 0
f-macro: 0.3118088716037848
f-micro: 0.4215192906781692
strange f score - 0.3986225467350215
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
AdaBoost DecisionTree max_depth = 2, n = 100: 
fit time = 6.35 21
predict time = 0.08333333333333333 5
f-macro: 0.3153888646236782
f-micro: 0.43829379343398034
strange f score - 0.4131220255348981


In [14]:
#countVectorizer
start_time = time.time()
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1, tokenizer=tokenize, stop_words=stop_words)
tfs4vec = vectorizer.fit_transform(token_list)
t = int(time.time() - start_time)
print ('TIME: {0}m {1}sec'.format(t / 60, t % 60))

TIME: 6.583333333333333m 35sec


In [21]:
df_test = module1.read_df_from_csv('ttk_test_etalon')

In [2]:
X_all, y_all = df_filter(_df = df, _tfs = tfs, corporation = 'beeline')
X4_all, y4_all = df_filter(_df = df, _tfs = tfs4vec, corporation = 'beeline')
X_test_all, y_test_all = df_filter(_df = df_test, _tfs = tfs, corporation = 'beeline')
X4_test_all, y4_test_all = df_filter(_df = df_test, _tfs = tfs4vec, corporation = 'beeline')
companies = ['mts', 'megafon', 'tele2', 'rostelecom']
for companie in companies:
    print('!!!!!!!!!!!!!!!!!')
    print(companie)
    print ('~~~~~~~~~~~~~~~~~')
    
    X, y = df_filter(_df = df, _tfs = tfs, corporation = companie)
    X_all = np.vstack((X_all, X))
    y_all = np.concatenate((y_all,y))
    X4, y4 = df_filter(_df = df, _tfs = tfs4vec, corporation = companie)
    X4_all = np.vstack((X4_all, X4))
    y4_all = np.concatenate((y4_all,y4))
    X_test, y_test = df_filter(_df = df_test, _tfs = tfs, corporation = companie)
    X_test_all = np.vstack((X_test_all, X_test))
    y_test_all = np.concatenate((y_test_all,y_test))
    X4_test, y4_test = df_filter(_df = df_test, _tfs = tfs4vec, corporation = companie)
    X4_test_all = np.vstack((X4_test_all, X4_test))
    y4_test_all = np.concatenate((y4_test_all,y4_test))
    

def calculation(X, y, X_test, y_test, X4, y4, X4_test, y4_test):    
    index = int((X.shape[0] + X_test.shape[0]) * 0.2)
    if (index < X_test.shape[0]):
        X = np.vstack((X, X_test[index:]))
        X_test = X_test[:index]
        y = np.concatenate((y,y_test[index:]))
        y_test = y_test[:index]
        X4 = np.vstack((X4, X4_test[index:]))
        X4_test = X4_test[:index]
        y4 = np.concatenate((y4,y4_test[index:]))
        y4_test = y4_test[:index]
    
    print ('DATA SIZE: TRAIN = {0}, TEST= {1}'.format(X.shape[0], X_test.shape[0]))
    print ('Naive Bayes: ')
    start_time = time.time()
    model = MultinomialNB()
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    t = int(time.time() - start_time)
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model = MultinomialNB()
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))
    print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print ('SVM: ')
    start_time = time.time()
    model = SVC()
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model = SVC()
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time) 
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))  
    print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print ('SVM linear kernel: ')
    start_time = time.time()
    model = SVC(kernel = 'linear')
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model = SVC(kernel = 'linear')
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time) 
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))
    print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print ('kNN 10: ')
    start_time = time.time()
    model = KNeighborsClassifier(n_neighbors=10)
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model =  KNeighborsClassifier(n_neighbors=10)
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time) 
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))
    print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print ('kNN 5: ')
    start_time = time.time()
    model = KNeighborsClassifier(n_neighbors=5)
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model =  KNeighborsClassifier(n_neighbors=5)
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time) 
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))
    print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print ('kNN 3: ')
    start_time = time.time()
    model = KNeighborsClassifier(n_neighbors=3)
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model =  KNeighborsClassifier(n_neighbors=3)
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time) 
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))
    print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print ('kNN 7: ')
    start_time = time.time()
    model = KNeighborsClassifier(n_neighbors=7)
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model =  KNeighborsClassifier(n_neighbors=7)
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time) 
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))
    print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print ('kNN 30: ')
    start_time = time.time()
    model = KNeighborsClassifier(n_neighbors=30)
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model =  KNeighborsClassifier(n_neighbors=30)
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time) 
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))
    print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print ('Decision Tree: ')
    start_time = time.time()
    model = DecisionTreeClassifier()
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model =  DecisionTreeClassifier()
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time) 
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))
    print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print ('Decision Tree max depth = 5: ')
    start_time = time.time()
    model = DecisionTreeClassifier(max_depth = 5)
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model =  DecisionTreeClassifier(max_depth = 5)
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time) 
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))
    print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print ('AdaBoost DecisionTree max_depth = 2, n = 100: ')
    start_time = time.time()
    model = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2), n_estimators=100,
    learning_rate=1)
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model =  AdaBoostClassifier(DecisionTreeClassifier(max_depth=2), n_estimators=100,
    learning_rate=1)
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time) 
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))

NameError: name 'df_filter' is not defined

In [1]:
calculation(X = X_all, y = y_all, X_test = X_test_all, y_test = y_test_all, X4 = X4_all, y4 = y4_all, X4_test = X4_test_all, y4_test = y4_test_all)

NameError: name 'calculation' is not defined